In [1]:
from IPython.display import HTML
HTML('''
    <style> body {font-family: "Roboto Condensed Light", "Roboto Condensed";} h2 {padding: 10px 12px; background-color: #E64626; position: static; color: #ffffff; font-size: 40px;} .text_cell_render p { font-size: 15px; } .text_cell_render h1 { font-size: 30px; } h1 {padding: 10px 12px; background-color: #E64626; color: #ffffff; font-size: 40px;} .text_cell_render h3 { padding: 10px 12px; background-color: #0148A4; position: static; color: #ffffff; font-size: 20px;} h4:before{
    content: "@"; font-family:"Wingdings"; font-style:regular; margin-right: 4px;} .text_cell_render h4 {padding: 8px; font-family: "Roboto Condensed Light"; position: static; font-style: italic; background-color: #FFB800; color: #ffffff; font-size: 18px; text-align: center; border-radius: 5px;}input[type=submit] {background-color: #E64626; border: solid; border-color: #734036; color: white; padding: 8px 16px; text-decoration: none; margin: 4px 2px; cursor: pointer; border-radius: 20px;}</style>
''')

# DATA 2001 Group Assignment
### LAB-07 : GROUP-07

In this notebook, we import various datasets to find a bustling score for every region in the SA2 Greater Sydney Region.

We start by importing all libraries and defining helper functions.
We have used the `folium` package for our interactive map. If it hasn't been installed, kindly run `pip install folium`

Furthermore, our Opal Taps dataset is relatively large (≈500mb) hence here is a link for the same in case too large to download off canvas
https://citydata.be.unsw.edu.au/dataset/opaldatansw_byloc_withsa4

In [2]:
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy import inspect
import psycopg2
import psycopg2.extras
import json
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
import folium
from shapely import wkt
from shapely.geometry import base
import shapely
import branca.colormap as cm
import plotly.express as px

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [3]:
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

We establish a connection to the server.

In [4]:
db, conn = pgconnect(credentials)

Connected successfully.


We ensure path is correct and have POSTGis enabled.

In [5]:
conn.execute(text("set search_path to public"))

In [6]:
conn.execute(text('''DROP EXTENSION IF EXISTS postgis;
CREATE EXTENSION postgis;
CREATE EXTENSION postgis_topology;'''))

In [7]:
query(conn, "select PostGIS_Version()")

,postgis_version
0,3.4 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


## Step 1: Importing, cleaning, and loading all data into SQL

### 1.1 Import SA2

We import our `SA2` files and do some initial cleaning for it.  

We set SRID to `4326` as this is consistent throughout our notebook.

In [8]:
srid = 4326

SA2 = gpd.read_file("SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")
SA2 = SA2[SA2['GCC_NAME21'] == 'Greater Sydney']

SA2['geom'] = SA2['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
SA2 = SA2.drop(columns="geometry")  # deleting the old copy
SA2.replace(['np', 'N/A', 'None', ''], np.nan, inplace=True)

Creating table and loading into SQL for SA2 file. We create a spatial index `idx_sa2geom` on the geom column as well the name column as these will be the values we most frequently access.

In [9]:
conn.execute(text("""
DROP TABLE IF EXISTS sa2_geom;
CREATE TABLE sa2_geom(
   sa2_code21 INT,
   sa2_name21 VARCHAR(100),
   chg_flag21 INT,
   chg_lbl21 VARCHAR(50),
   sa3_code21 INT,
   sa3_name21 VARCHAR(50),
   sa4_code21 INT,
   sa4_name21 VARCHAR(50),
   gcc_code21 VARCHAR(10),
   gcc_name21 VARCHAR(100),
   ste_code21 INT,
   ste_name21 VARCHAR(50),
   aus_code21 VARCHAR(10),
   aus_name21 VARCHAR(50),
   areasqkm21 DECIMAL,
   loci_uri21 VARCHAR(200),
   geom GEOMETRY(MULTIPOLYGON, 4326)
);"""))
SA2.columns = map(str.lower, SA2.columns)
SA2.to_sql("sa2_geom", conn, if_exists='replace', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
conn.execute(text("""
CREATE INDEX idx_sa2geom
ON sa2_geom USING GIST(geom) INCLUDE (sa2_name21);
"""))

We can see the data has been loaded into the table called <u><strong>sa2_geom</strong></u>.


In [10]:
query(conn, "SELECT * FROM sa2_geom")

,sa2_code21,sa2_name21,chg_flag21,chg_lbl21,sa3_code21,sa3_name21,sa4_code21,sa4_name21,gcc_code21,gcc_name21,ste_code21,ste_name21,aus_code21,aus_name21,areasqkm21,loci_uri21,geom
0,102011028,Avoca Beach - Copacabana,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,6.4376,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E6100000010000000103000000010000005E...
1,102011029,Box Head - MacMasters Beach,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,32.0802,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E61000000100000001030000000100000010...
2,102011030,Calga - Kulnura,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,767.9512,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E61000000200000001030000000100000085...
3,102011031,Erina - Green Point,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,33.7934,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E61000000100000001030000000100000041...
4,102011032,Gosford - Springfield,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,16.9123,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E6100000010000000103000000010000007E...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,128021537,Royal National Park,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,139.3336,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E61000000100000001030000000100000046...
369,128021538,Sutherland - Kirrawee,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,7.7550,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E61000000100000001030000000100000089...
370,128021607,Engadine,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,8.9538,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E6100000010000000103000000010000008E...
371,128021608,Loftus - Yarrawarrah,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,3.8436,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E610000001000000010300000001000000A1...


### 1.2 Import Catchments

Importing, cleaning, and loading `catchment` files. The catchment folder includes shapefiles for primary, secondary, and future catchments. We only take into consideration the primary and secondary. (More details in report)

We merge both primary and secondary catchments into a single table to record all catchments.

We set SRID to `4326` as this is consistent throughout our notebook.

In [11]:
srid = 4326

catchments_primary = gpd.read_file("catchments/catchments_primary.shp")
catchments_secondary = gpd.read_file("catchments/catchments_secondary.shp")

catchments_primary.replace(['np', 'N/A', 'None', ''], np.nan, inplace=True)
catchments_secondary.replace(['np', 'N/A', 'None', ''], np.nan, inplace=True)

catchments_all = pd.concat([catchments_primary, catchments_secondary], axis=0)
catchments_all['geom'] = catchments_all['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
catchments_all = catchments_all.drop(columns="geometry")  # deleting the old copy
catchments_all.columns = map(str.lower, catchments_all.columns)

In [12]:
conn.execute(text("""
DROP TABLE IF EXISTS catchments;
CREATE TABLE catchments(
   use_id INT,
   catch_type VARCHAR(100),
   use_desc VARCHAR(50),
   add_date INT,
   kindergart CHAR,
   year1 CHAR,
   year2 CHAR,
   year3 CHAR,
   year4 CHAR,
   year5 CHAR,
   year6 CHAR,
   year7 CHAR,
   year8 CHAR,
   year9 CHAR,
   year10 CHAR,
   year11 CHAR,
   year12 CHAR,
   priority VARCHAR(50),
   geom GEOMETRY(MULTIPOLYGON, 4326)
);"""))
catchments_all.to_sql("catchments", conn, if_exists='replace', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})

98

We can see the data has been loaded into the table called <u><strong>catchments</strong></u>.


In [13]:
query(conn, "SELECT * FROM catchments")

,use_id,catch_type,use_desc,add_date,kindergart,year1,year2,year3,year4,year5,year6,year7,year8,year9,year10,year11,year12,priority,geom
0,2838,PRIMARY,Parklea PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020E61000000100000001030000000100000078...
1,2404,PRIMARY,Lindfield EPS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020E610000001000000010300000001000000BE...
2,4393,PRIMARY,Carlingford WPS,20220223,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020E61000000100000001030000000100000065...
3,7308,PRIMARY,Plattsburg PS,20200723,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020E6100000010000000103000000010000003D...
4,4615,PRIMARY,Caddies Ck PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020E61000000100000001030000000100000056...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093,8213,HIGH_BOYS,Birrong BHS,20211221,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,0106000020E61000000100000001030000000100000040...
2094,8108,HIGH_COED,Cessnock HS,20230405,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,0106000020E610000001000000010300000001000000AD...
2095,3235,CENTRAL_HIGH,Tooleybuc CS,20200512,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,0106000020E6100000010000000103000000010000003E...
2096,1115,CENTRAL_HIGH,Balranald CS,20200512,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,0106000020E6100000010000000103000000010000005B...


### 1.3 Import Businesses

Importing, cleaning, and loading `businesses` data.

In [14]:
businesses = pd.read_csv("Businesses.csv")
businesses.columns = map(str.lower, businesses.columns)

businesses.replace(['np', 'N/A', 'None', ''], np.nan, inplace=True)

conn.execute(text("""
DROP TABLE IF EXISTS Businesses;
CREATE TABLE Businesses(
    industry_code CHAR,
    industry_name VARCHAR(100),
    sa2_code INT,
    sa2_name VARCHAR(100),
    "0_to_50k_businesses" INT,
    "50k_to_200k_businesses" INT,
    "200k_to_2m_businesses" INT,
    "2m_to_5m_businesses" INT,
    "5m_to_10m_businesses" INT,
    "10m_or_more_businesses" INT,
    total_businesses INT
);"""))

businesses.to_sql("businesses", con=conn, if_exists='replace', index=False)


217

We can see the data has been loaded into the table called <u><strong>businesses</strong></u>.


In [15]:
query(conn, "SELECT * FROM businesses")

,industry_code,industry_name,sa2_code,sa2_name,0_to_50k_businesses,50k_to_200k_businesses,200k_to_2m_businesses,2m_to_5m_businesses,5m_to_10m_businesses,10m_or_more_businesses,total_businesses
0,A,"Agriculture, Forestry and Fishing",101021007,Braidwood,136,92,63,4,0,0,296
1,A,"Agriculture, Forestry and Fishing",101021008,Karabar,6,3,0,0,0,0,9
2,A,"Agriculture, Forestry and Fishing",101021009,Queanbeyan,6,4,3,0,0,3,15
3,A,"Agriculture, Forestry and Fishing",101021010,Queanbeyan - East,0,3,0,0,0,0,3
4,A,"Agriculture, Forestry and Fishing",101021012,Queanbeyan West - Jerrabomberra,7,4,5,0,0,0,16
...,...,...,...,...,...,...,...,...,...,...,...
12212,S,Other Services,128021538,Sutherland - Kirrawee,21,66,58,3,3,0,152
12213,S,Other Services,128021607,Engadine,13,41,31,3,0,0,87
12214,S,Other Services,128021608,Loftus - Yarrawarrah,0,10,10,0,0,0,22
12215,S,Other Services,128021609,Woronora Heights,0,3,5,0,0,0,9


### 1.4 Import Stops

Importing, cleaning, and loading `stops` data.

We set SRID to `4326` as this is consistent throughout our notebook.

In [16]:
srid = 4326

stops = pd.read_csv("Stops.txt")
stops['geom'] = gpd.points_from_xy(stops.stop_lon, stops.stop_lat)  # creating the geometry column
stops = stops.drop(columns=['stop_lat', 'stop_lon'])  # removing the old latitude/longitude fields
stops['geom'] = stops['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))
stops.replace(['np', 'N/A', 'None', ''], np.nan, inplace=True)

In [17]:
stops.columns = map(str.lower, stops.columns)

conn.execute(text("""
DROP TABLE IF EXISTS Stops;
CREATE TABLE Stops(
   stop_id VARCHAR(50) PRIMARY KEY,
   stop_code INT,
   stop_name VARCHAR(100),
   location_type INT,
   parent_station VARCHAR(50),
   wheelchair_boarding INT,
   platform_code VARCHAR(50),
   geom GEOMETRY(POINT, 4326)
);"""))

stops.to_sql("stops", con=conn, if_exists='replace', index=False, dtype={'geom': Geometry('POINT', srid)})

718

We can see the data has been loaded into the table called <u><strong>stops</strong></u>.


In [18]:
query(conn, "SELECT * FROM stops")

,stop_id,stop_code,stop_name,location_type,parent_station,wheelchair_boarding,platform_code,geom
0,200039,200039.0,"Central Station, Eddy Av, Stand A",NaN,200060,0,None,0101000020E6100000FFA631FF9CE66240A1FF6524ECF0...
1,200054,200054.0,"Central Station, Eddy Av, Stand D",NaN,200060,0,None,0101000020E61000002F928BAC9FE66240E33DC7C1E6F0...
2,200060,NaN,Central Station,1.0,None,0,None,0101000020E6100000817FA2F299E662408FF33DAC29F1...
3,201510,NaN,Redfern Station,1.0,None,0,None,0101000020E61000009E57611C5DE6624060304CE622F2...
4,201646,201646.0,"Redfern Station, Gibbons St, Stand B",NaN,201510,0,None,0101000020E6100000DBF9333D5DE662403DFA6B9D58F2...
...,...,...,...,...,...,...,...,...
114713,212753,212753.0,"Sydney Olympic Park Wharf, Side B",NaN,21271,1,B,0101000020E6100000AF9B3D8185E262408F52D7D537E9...
114714,2137185,2137185.0,"Cabarita Wharf, Side A",NaN,21371,1,1A,0101000020E6100000EB409ADCBDE3624089CE4C0B9BEB...
114715,2137186,2137186.0,"Cabarita Wharf, Side B",NaN,21371,1,1B,0101000020E6100000C4F9BEA2BDE362403EB375529EEB...
114716,21501,21501.0,Parramatta Wharf,NaN,2150112,1,None,0101000020E6100000E443E4A456E0624025C1A4032EE8...


### 1.5 Import Polling Places

Importing, cleaning, and loading `polling place` data.

We set SRID to `4326` as this is consistent throughout our notebook.

In [19]:
srid = 4326

pollingPlaces = pd.read_csv("PollingPlaces2019.csv")
pollingPlaces.columns = map(str.lower, pollingPlaces.columns)
pollingPlaces.replace(['np', 'N/A', 'None', ''], np.nan, inplace=True)

pollingPlaces['the_geom'] = gpd.points_from_xy(pollingPlaces.longitude, pollingPlaces.latitude)  # creating the geometry column
pollingPlaces['the_geom'] = pollingPlaces['the_geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

In [20]:
conn.execute(text("""
DROP TABLE IF EXISTS polling_places;
CREATE TABLE polling_Places(
   FID VARCHAR(100) PRIMARY KEY,
   state VARCHAR(10),
   division_id INT,
   division_name VARCHAR(50),
   polling_place_id INT,
   polling_place_type_id INT,
   polling_place_name VARCHAR(100),
   premises_name VARCHAR(100),
   premises_address_1 VARCHAR(100),
   premises_address_2 VARCHAR(100),
   premises_address_3 VARCHAR(100),
   premises_suburb VARCHAR(100),
   premises_state_abbreviation VARCHAR(10),
   premises_post_code INT,
   latitude DECIMAL,
   longitude DECIMAL,
   the_geom GEOMETRY(POINT, 4326)
);"""))

pollingPlaces.to_sql("polling_places", con=conn, if_exists='replace', index=False, dtype={'the_geom': Geometry('POINT', srid)})


930

We can see the data has been loaded into the table called <u><strong>polling_places</strong></u>.


In [21]:
query(conn, "SELECT * FROM polling_places")

,fid,state,division_id,division_name,polling_place_id,polling_place_type_id,polling_place_name,premises_name,premises_address_1,premises_address_2,premises_address_3,premises_suburb,premises_state_abbreviation,premises_post_code,latitude,longitude,the_geom
0,aec_federal_election_polling_places_2019.fid-4...,NSW,104,Barton,33595,2,Special Hospital Team 1,Multiple sites,None,None,None,None,NSW,NaN,NaN,NaN,0101000020E6100000000000000000F87F000000000000...
1,aec_federal_election_polling_places_2019.fid-4...,NSW,105,Bennelong,33596,2,Special Hospital Team 1,Multiple sites,None,None,None,None,NSW,NaN,NaN,NaN,0101000020E6100000000000000000F87F000000000000...
2,aec_federal_election_polling_places_2019.fid-4...,NSW,107,Blaxland,33600,2,Special Hospital Team 1,Multiple sites,None,None,None,None,NSW,NaN,NaN,NaN,0101000020E6100000000000000000F87F000000000000...
3,aec_federal_election_polling_places_2019.fid-4...,NSW,109,Calare,33603,2,Special Hospital Team 1,Multiple sites,None,None,None,ORANGE,NSW,2800.0,NaN,NaN,0101000020E6100000000000000000F87F000000000000...
4,aec_federal_election_polling_places_2019.fid-4...,NSW,113,Cowper,33716,2,Special Hospital Team 2,Multiple sites,None,None,None,None,NSW,NaN,NaN,NaN,0101000020E6100000000000000000F87F000000000000...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,2809,1,Warilla South,Warilla High School,10 Keross Ave,None,None,BARRACK HEIGHTS,NSW,2528.0,-34.564200,150.858000,0101000020E6100000FA7E6ABC74DB62409C33A2B43748...
2926,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,58798,5,Warilla WHITLAM PPVC,2/144 Shellharbour Rd,None,None,None,WARILLA,NSW,2528.0,-34.550823,150.859755,0101000020E6100000BD32141C83DB624011F28B5C8146...
2927,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,31242,1,Welby,Welby Community Hall,14 Currockbilly St,None,None,WELBY,NSW,2575.0,-34.440900,150.424000,0101000020E610000021B0726891CD6240386744696F38...
2928,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,564,1,Windang,Windang Public School,60-64 Oakland Ave,None,None,WINDANG,NSW,2528.0,-34.531600,150.866000,0101000020E6100000C1CAA145B6DB6240DC4603780B44...


### 1.6 Import Population

Importing, cleaning and loading `population` data. We rename all the columns to then feed into our table. We also create an additional column called youth population which we will use for later calculations. We also create an index `idx_sa2_code_name` for faster retrievals in future queries.

In [22]:
population = pd.read_csv("Population.csv")
new_columns = {
    '0-4_people': 'people_0_4',
    '5-9_people': 'people_5_9',
    '10-14_people': 'people_10_14',
    '15-19_people': 'people_15_19',
    '20-24_people': 'people_20_24',
    '25-29_people': 'people_25_29',
    '30-34_people': 'people_30_34',
    '35-39_people': 'people_35_39',
    '40-44_people': 'people_40_44',
    '45-49_people': 'people_45_49',
    '50-54_people': 'people_50_54',
    '55-59_people': 'people_55_59',
    '60-64_people': 'people_60_64',
    '65-69_people': 'people_65_69',
    '70-74_people': 'people_70_74',
    '75-79_people': 'people_75_79',
    '80-84_people': 'people_80_84',
    '85-and-over_people': 'people_85_over',
    'total_people': 'total_people'  # Assuming this name is already correct
}
population.rename(columns=new_columns, inplace=True)
population.replace(['np', 'N/A', 'None', '', 0], np.nan, inplace=True)
population['youth_population'] = (
    population['people_0_4'] +
    population['people_5_9'] +
    population['people_10_14'] +
    population['people_15_19']
)

In [23]:
conn.execute(text("""
DROP TABLE IF EXISTS Population;
CREATE TABLE Population(
    sa2_code INT,
    sa2_name VARCHAR(100),
    people_0_4 INT,
    people_5_9 INT,
    people_10_14 INT,
    people_15_19 INT,
    people_20_24 INT,
    people_25_29 INT,
    people_30_34 INT,
    people_35_39 INT,
    people_40_44 INT,
    people_45_49 INT,
    people_50_54 INT,
    people_55_59 INT,
    people_60_64 INT,
    people_65_69 INT,
    people_70_74 INT,
    people_75_79 INT,
    people_80_84 INT,
    people_85_over INT,
    total_people INT,
    youth_population INT
);"""))

population.to_sql("population", con=conn, if_exists='replace', index=False)
conn.execute(text("""
CREATE INDEX idx_sa2_code_name
ON population(sa2_code, sa2_name);
"""))


We can see the data has been loaded into the table called <u><strong>population</strong></u>.


In [24]:
query(conn, "SELECT * FROM population")

,sa2_code,sa2_name,people_0_4,people_5_9,people_10_14,people_15_19,people_20_24,people_25_29,people_30_34,people_35_39,...,people_50_54,people_55_59,people_60_64,people_65_69,people_70_74,people_75_79,people_80_84,people_85_over,total_people,youth_population
0,102011028,Avoca Beach - Copacabana,424.0,522.0,623.0,552.0,386.0,222.0,306.0,416.0,...,602.0,570.0,520.0,464.0,369.0,226.0,142.0,70.0,7530.0,2121.0
1,102011029,Box Head - MacMasters Beach,511.0,666.0,702.0,592.0,461.0,347.0,420.0,535.0,...,749.0,794.0,895.0,863.0,925.0,603.0,331.0,264.0,11052.0,2471.0
2,102011030,Calga - Kulnura,200.0,225.0,258.0,278.0,274.0,227.0,214.0,286.0,...,436.0,422.0,397.0,327.0,264.0,190.0,100.0,75.0,4748.0,961.0
3,102011031,Erina - Green Point,683.0,804.0,880.0,838.0,661.0,502.0,587.0,757.0,...,882.0,901.0,930.0,917.0,1065.0,976.0,773.0,1028.0,14803.0,3205.0
4,102011032,Gosford - Springfield,1164.0,1044.0,1084.0,1072.0,1499.0,1864.0,1750.0,1520.0,...,1241.0,1377.0,1285.0,1166.0,949.0,664.0,476.0,537.0,21346.0,4364.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,128021537,Royal National Park,2.0,4.0,10.0,4.0,9.0,7.0,1.0,2.0,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,45.0,20.0
369,128021538,Sutherland - Kirrawee,1519.0,1292.0,1150.0,1117.0,1335.0,1852.0,2120.0,1945.0,...,1391.0,1285.0,1157.0,909.0,909.0,781.0,582.0,807.0,23369.0,5078.0
370,128021607,Engadine,1157.0,1283.0,1469.0,1209.0,891.0,675.0,928.0,1229.0,...,1086.0,909.0,764.0,707.0,886.0,748.0,389.0,327.0,17379.0,5118.0
371,128021608,Loftus - Yarrawarrah,503.0,487.0,575.0,508.0,380.0,293.0,426.0,493.0,...,477.0,450.0,387.0,418.0,335.0,263.0,192.0,109.0,7354.0,2073.0


### 1.7 Import Income

Importing, cleaning and loading `income` data. We will use this data later for <u><strong>correlation analysis</strong></u>.

In [25]:
income = pd.read_csv("Income.csv")
income.columns = map(str.lower, income.columns)
income.replace(['np', 'N/A', 'None', ''], np.nan, inplace=True)

In [26]:
conn.execute(text("""
DROP TABLE IF EXISTS Income;
CREATE TABLE Income(
   sa2_code21 INT PRIMARY KEY NOT NULL,
   sa2_name VARCHAR(50),
   earners INT,
   median_age INT,
   median_income INT,
   mean_income INT
);"""))

income.to_sql("income", con=conn, if_exists='replace', index=False)

642

We can see the data has been loaded into the table called <u><strong>income</strong></u>.

We then create a table which only looks at regions in greater sydney called <u><strong>income_greater_sydney</strong></u>.

In [27]:
query(conn, """
DROP TABLE IF EXISTS income_greater_sydney;
CREATE TABLE income_greater_sydney
AS
(SELECT *
FROM income
WHERE sa2_code21 IN (SELECT CAST(sa2_code21 AS BIGINT) FROM sa2_geom));
SELECT * FROM income_greater_sydney
""")

,sa2_code21,sa2_name,earners,median_age,median_income,mean_income
0,102011028,Avoca Beach - Copacabana,4749,47,55065,77615
1,102011029,Box Head - MacMasters Beach,6636,49,51927,71509
2,102011030,Calga - Kulnura,2965,49,49168,63802
3,102011031,Erina - Green Point,8010,48,51905,71992
4,102011032,Gosford - Springfield,12051,41,54372,65283
...,...,...,...,...,...,...
368,128021537,Royal National Park,14,37,36980,47584
369,128021538,Sutherland - Kirrawee,13895,41,64940,74867
370,128021607,Engadine,10239,43,63695,72995
371,128021608,Loftus - Yarrawarrah,4424,45,63087,76440


## Step 2: We start performing queries on our tables to analyze the data

First, we drop any regions with population under 100 from `sa2_geom`

In [28]:
conn.execute(text("""
DELETE FROM sa2_geom
WHERE CAST(sa2_code21 AS BIGINT) IN (
    SELECT sa2_code
    FROM population
    WHERE total_people < 100
);
"""))

### 2.1 Businesses

Calculating the number of `businesses` per region per 1000 people.

In [29]:
conn.execute(text( """
DROP TABLE IF EXISTS businesses_per_1000_people;
CREATE TABLE businesses_per_1000_people
AS
(SELECT
    sa2_geom.SA2_NAME21,
    COUNT(businesses.sa2_code)/(total_people/1000.0) AS businesses_per_1000
FROM
    sa2_geom
JOIN
    businesses
ON
    businesses.sa2_code = CAST(sa2_code21 AS bigint)
JOIN
    population
ON
    population.sa2_code = CAST(sa2_code21 AS bigint)
WHERE
     businesses.industry_name = 'Retail Trade' OR businesses.industry_name = 'Arts and Recreation Services'

GROUP BY
    sa2_geom.SA2_NAME21, total_people);
"""))
query(conn, "SELECT * FROM businesses_per_1000_people")

,sa2_name21,businesses_per_1000
0,Surry Hills,0.124587
1,Hurstville - Central,0.164704
2,Illawong - Alfords Point,0.189233
3,Blue Haven - San Remo,0.175608
4,Elderslie - Narellan,0.142045
...,...,...
356,Ingleburn,0.119211
357,Campbelltown - Woodbine,0.089839
358,Blacktown (North) - Marayong,0.096427
359,Pemulwuy - Greystanes (North),0.183368


Creating Z-value for businesses and store it in a table called <u><strong> businesses_zscore </strong></u>.

In [30]:
query(conn, """
DROP TABLE IF EXISTS businesses_zscore;
CREATE TABLE businesses_zscore
AS
(SELECT
    sa2_name21,
    (businesses_per_1000 - (SELECT AVG(businesses_per_1000) FROM businesses_per_1000_people)) /
    (SELECT STDDEV(businesses_per_1000) FROM businesses_per_1000_people) AS z_score
FROM
    businesses_per_1000_people);
SELECT * FROM businesses_zscore;
""")

,sa2_name21,z_score
0,Surry Hills,-0.238094
1,Hurstville - Central,-0.062384
2,Illawong - Alfords Point,0.045052
3,Blue Haven - San Remo,-0.014624
4,Elderslie - Narellan,-0.161627
...,...,...
356,Ingleburn,-0.261643
357,Campbelltown - Woodbine,-0.390290
358,Blacktown (North) - Marayong,-0.361434
359,Pemulwuy - Greystanes (North),0.019367


### 2.2 Stops

Finding number of `public transport stops` in each neighborhood.

In [31]:
conn.execute(text( """
DROP TABLE IF EXISTS number_of_stops;
CREATE TABLE number_of_stops
AS
(SELECT
    sa2_geom.SA2_NAME21,
    COUNT(stops.stop_id) as number
FROM
    sa2_geom
LEFT JOIN
    stops
ON
    ST_Intersects(sa2_geom.geom, stops.geom)

GROUP BY
    sa2_geom.SA2_NAME21);
"""))
query(conn, "SELECT * FROM number_of_stops")

,sa2_name21,number
0,Toukley - Norah Head,174
1,Warwick Farm,54
2,Edensor Park,77
3,Bexley - South,149
4,Pitt Town - McGraths Hill,268
...,...,...
356,Beacon Hill - Narraweena,145
357,Woy Woy - Blackwall,305
358,Umina - Booker Bay - Patonga,387
359,St Clair,177


Creating Z-value for stops and store it in a table called <u><strong> stops_zscore </strong></u>.

In [32]:
query(conn, """
DROP TABLE IF EXISTS stops_zscore;
CREATE TABLE stops_zscore
AS
(SELECT
    sa2_name21,
    (number - (SELECT AVG(number) FROM number_of_stops)) /
    (SELECT STDDEV(number) FROM number_of_stops) AS z_score
FROM
    number_of_stops);
SELECT * FROM stops_zscore;
""")

,sa2_name21,z_score
0,Toukley - Norah Head,0.245824
1,Warwick Farm,-1.163533
2,Edensor Park,-0.893406
3,Bexley - South,-0.047792
4,Pitt Town - McGraths Hill,1.349821
...,...,...
356,Beacon Hill - Narraweena,-0.094770
357,Woy Woy - Blackwall,1.784373
358,Umina - Booker Bay - Patonga,2.747433
359,St Clair,0.281058


### 2.3 Polling Places

Finding number of `polls` in each location.

In [33]:
query(conn, """
DROP TABLE IF EXISTS number_of_polls;
CREATE TABLE number_of_polls
AS
(SELECT
    sa2_geom.SA2_NAME21,
    COUNT(polling_places.polling_place_id) as number
FROM
    sa2_geom
LEFT JOIN
    polling_places
ON
    ST_Intersects(sa2_geom.geom, polling_places.the_geom)

GROUP BY
    sa2_geom.SA2_NAME21);
SELECT * FROM number_of_polls
""")

,sa2_name21,number
0,Acacia Gardens,1
1,Annandale (NSW),4
2,Arncliffe - Bardwell Valley,5
3,Artarmon,2
4,Ashcroft - Busby - Miller,4
...,...,...
356,Wyoming,5
357,Wyong,6
358,Yagoona - Birrong,3
359,Yarramundi - Londonderry,3


Creating Z-value for polling locations and store it in a table called <u><strong> polls_zscore </strong></u>.

In [34]:
query(conn, """
DROP TABLE IF EXISTS polls_zscore;
CREATE TABLE polls_zscore
AS
(SELECT
    sa2_name21,
    (number - (SELECT AVG(number) FROM number_of_polls)) /
    (SELECT STDDEV(number) FROM number_of_polls) AS z_score
FROM
    number_of_polls);
SELECT * FROM polls_zscore;
""")


,sa2_name21,z_score
0,Acacia Gardens,-0.775497
1,Annandale (NSW),-0.048972
2,Arncliffe - Bardwell Valley,0.193203
3,Artarmon,-0.533322
4,Ashcroft - Busby - Miller,-0.048972
...,...,...
356,Wyoming,0.193203
357,Wyong,0.435378
358,Yagoona - Birrong,-0.291147
359,Yarramundi - Londonderry,-0.291147


### 2.4 Catchments

Finding number of `catchments` per 1000 young people per location.

In [35]:
query(conn, """
DROP TABLE IF EXISTS catchments_per_1000_people;
CREATE TABLE catchments_per_1000_people
AS
(SELECT
    sa2_geom.SA2_NAME21,
    COUNT(catchments.use_id)/(youth_population/1000.0) AS catchments_per_1000
FROM
    sa2_geom
JOIN
    catchments
ON
    ST_Intersects(sa2_geom.geom, catchments.geom)
JOIN
    population
ON
    population.sa2_code = CAST(sa2_code21 AS bigint)
GROUP BY
    sa2_geom.SA2_NAME21, youth_population);
SELECT * FROM catchments_per_1000_people
""")


,sa2_name21,catchments_per_1000
0,Kellyville - West,3.486529
1,Wentworth Point - Sydney Olympic Park,3.141690
2,Glendenning - Dean Park,5.544554
3,Bossley Park - Abbotsbury,2.685396
4,Carlingford - West,2.561072
...,...,...
356,Greenwich - Riverview,2.793296
357,Heathcote - Waterfall,4.759072
358,Croydon,4.917300
359,Berowra - Brooklyn - Cowan,3.315250


Creating Z-value for catchments and store it in a table called <u><strong> catchments_zscore </strong></u>.

In [36]:
query(conn, """
DROP TABLE IF EXISTS catchments_zscore;
CREATE TABLE catchments_zscore
AS
(SELECT
    sa2_name21,
    (catchments_per_1000 - (SELECT AVG(catchments_per_1000) FROM catchments_per_1000_people)) /
    (SELECT STDDEV(catchments_per_1000) FROM catchments_per_1000_people) AS z_score
FROM
    catchments_per_1000_people);
SELECT * FROM catchments_zscore;
""")

,sa2_name21,z_score
0,Kellyville - West,-0.124296
1,Wentworth Point - Sydney Olympic Park,-0.180776
2,Glendenning - Dean Park,0.212787
3,Bossley Park - Abbotsbury,-0.255513
4,Carlingford - West,-0.275876
...,...,...
356,Greenwich - Riverview,-0.237840
357,Heathcote - Waterfall,0.084133
358,Croydon,0.110049
359,Berowra - Brooklyn - Cowan,-0.152349


### Calculating Sigmoid Values

We have formed 4 new tables above, storing a z-value for each metric. We can now join these tables and find the sigmoid value for each region by using the sigmoid function.

We store the data in a table called <u><strong> sigmoid_values </strong></u>.

In [37]:
conn.execute(text( """
DROP TABLE IF EXISTS sigmoid_values;
CREATE TABLE sigmoid_values
AS
(SELECT
    sa2_name21,
    1.0 / (1.0 + EXP(- (b.z_score + s.z_score + p.z_score + c.z_score))) AS sigmoid_value
FROM
    businesses_zscore b
JOIN
    stops_zscore s USING(sa2_name21)
JOIN
    stops_zscore p USING(sa2_name21)
JOIN
    catchments_zscore c USING(sa2_name21));
"""))


We have a table which displays the bustling metric for all SA2 regions.

In [38]:
query(conn, """
SELECT sa2_name21 AS Region, sigmoid_value AS Bustling_Score
FROM sigmoid_values
WHERE sigmoid_values IS NOT NULL;
""")

,region,bustling_score
0,Acacia Gardens,0.267445
1,Annandale (NSW),0.072999
2,Arncliffe - Bardwell Valley,0.130452
3,Artarmon,0.063944
4,Ashcroft - Busby - Miller,0.722083
...,...,...
355,Wyoming,0.514731
356,Wyong,0.717923
357,Yagoona - Birrong,0.708104
358,Yarramundi - Londonderry,0.574689


## Step 3: Extension
In extension, we will source two more datasets and use these calculate a more precise bustling score metric for each region.

### 3.1 Opal Data

We will start by importing our first extension spatial dataset. It holds data for various Opal Card <u><strong>tap-ons</strong></u> and <u><strong>tap-offs</strong></u> on public transport. We also calculate a Spatial value from the longitude and latitude given.

In [39]:
sa4_tap = gpd.read_file("opaldatansw_byloc_withsa4")

sa4_tap = sa4_tap[sa4_tap['Region'] == 'Greater Sydney']

sa4_tap.columns = map(str.lower, sa4_tap.columns)
sa4_tap = sa4_tap.rename(columns={'date': 'dateid'})
sa4_tap['geom'] = sa4_tap['geometry'].apply(lambda x: create_wkt_element(x, 4326))
sa4_tap.drop(columns='geometry', inplace=True)

DriverError: opaldatansw_byloc_withsa4: No such file or directory

In [ ]:
conn.execute(text("""
DROP TABLE IF EXISTS tap_data;
CREATE TABLE tap_data(
    fid INT PRIMARY KEY,
    target_f_1 INT,
    location_n VARCHAR(255),
    tsn INT,
    mode VARCHAR(50),
    dateid INT,
    weekday VARCHAR(50),
    tap CHAR(3),
    time VARCHAR(50),
    count INT,
    region VARCHAR(255),
    sa4_name_1 VARCHAR(255),
    first_sa41 INT,
    geom GEOMETRY(POINT, 4326)
);
"""))
sa4_tap.to_sql('tap_data', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', 4326)})

In [ ]:
query(conn, "SELECT * FROM tap_data")

We can see the data has been loaded into the table called <u><strong>tap_data</strong></u>.

Since the counts of taps are seperated on increments of 15 mins, we aggregate the counts of taps for each stop.


In [ ]:
conn.execute(text('''
DROP TABLE IF EXISTS aggregated_tap_events;
CREATE TABLE aggregated_tap_events AS
SELECT
    geom,
    ST_AsText(geom) AS geom_text,
    SUM(count) AS total_taps,
    mode,
    region,
    sa4_name_1,
    first_sa41
FROM
    tap_data
GROUP BY
    geom, mode, region, sa4_name_1, first_sa41;
'''))

Calculating stops for each stop and store it in a table called <u><strong> aggregated_tap_events </strong></u>.

In [ ]:
query(conn, "SELECT * FROM aggregated_tap_events")

We now use a spatial join to find the number of `taps` in each region in our SA2 table. We also calculate the z-score.

In [ ]:
conn.execute(text( '''
DROP TABLE IF EXISTS tap_summary_per_sa2;
CREATE TABLE tap_summary_per_sa2 AS
SELECT
    SA2_NAME21,
    total_taps,
    CASE
        WHEN stddev_total_taps = 0 THEN 0
        ELSE (total_taps - avg_total_taps) / stddev_total_taps
    END AS z_score
FROM
    (
        SELECT
            SA2_geom.SA2_NAME21,
            COALESCE(SUM(aggregated_tap_events.total_taps), 0) AS total_taps,
            AVG(SUM(aggregated_tap_events.total_taps)) OVER () AS avg_total_taps,
            STDDEV_SAMP(SUM(aggregated_tap_events.total_taps)) OVER () AS stddev_total_taps
        FROM
            SA2_geom
        LEFT JOIN
            aggregated_tap_events
        ON
            ST_Intersects(SA2_geom.geom, aggregated_tap_events.geom)
        GROUP BY
            SA2_geom.SA2_NAME21

    ) AS derived_table
ORDER BY
    total_taps DESC;
'''
))

Calculating taps for each region and store it in a table called <u><strong> tap_summary_per_sa2 </strong></u>.

In [ ]:
query(conn, "SELECT * FROM tap_summary_per_sa2")

### 3.2 Population Change Data

Now importing the data set for population change

In [ ]:
regionalpop = gpd.read_file("au-govt-abs-abs-regional-population-sa2-2001-2021-sa2-2016(nsw).json")
regionalpop = regionalpop[regionalpop['gccsa_name_2016'] == 'Greater Sydney']
regionalpop['geom'] = regionalpop['geometry'].apply(lambda x: create_wkt_element(x, srid=4326))
regionalpop = regionalpop.drop(columns=['geometry'])

In [ ]:
conn.execute(text("""
DROP TABLE IF EXISTS regional_population;
CREATE TABLE regional_population(
    id VARCHAR(255) PRIMARY KEY,
    primaryindex INT,
    state_code_2016 INT,
    state_name_2016 VARCHAR(50),
    gccsa_code_2016 INT,
    gccsa_name_2016 VARCHAR(50),
    sa4_code_2016 INT,
    sa4_name_2016 varCHAR(50),
    sa3_code_2016 INT,
    sa3_name_2016 VARCHAR(50),
    births_2020_21 INT,
    deaths_2020_21 INT,
    natural_increase_2020_21 INT,
    internal_arrivals_2020_21 INT,
    internal_departures_2020_21 INT,
    net_internal_migration_2020_21 INT,
    overseas_arrivals_2020_21 INT,
    overseas_departures_2020_21 INT,
    net_overseas_migration_2020_21 INT,
    geom GEOMETRY(MULTIPOLYGON,4326)
);
"""))
regionalpop.to_sql('regional_population', conn, if_exists='replace', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid=4326)})

We can see the data has been loaded into the table called <u><strong>regional_population</strong></u>.

In [ ]:
query(conn,'select * from regional_population ')

We calculate the net change by taking into account the number of all the additions and removals from a population. We then perform spatial join to find the data per region.

In [ ]:
conn.execute(text('''ALTER TABLE regional_population ADD COLUMN IF NOT EXISTS net_added_2020_2021 INT;
UPDATE regional_population
SET net_added_2020_2021 = COALESCE(births_2020_21, 0) + COALESCE(internal_arrivals_2020_21, 0) + COALESCE(overseas_arrivals_2020_21, 0) - COALESCE(deaths_2020_21, 0) - COALESCE(internal_departures_2020_21, 0) - COALESCE(overseas_departures_2020_21, 0);

DROP TABLE IF EXISTS sa2_with_net_added;
CREATE TABLE sa2_with_net_added AS
SELECT
    sa2.sa2_name21,
    CAST(AVG(COALESCE(rp.net_added_2020_2021, 0)) AS INT) AS avg_net_added_2020_2021
FROM
    sa2_geom AS sa2
LEFT JOIN
    regional_population AS rp
ON
    ST_Intersects(sa2.geom, rp.geom)
GROUP BY
    sa2.sa2_name21, sa2.geom;
'''))

Calculating z scores for `population change` per region.

In [ ]:
conn.execute(text( """
DROP TABLE IF EXISTS population_change_zscore;
CREATE TABLE population_change_zscore
AS
(SELECT
    sa2_name21,
    (avg_net_added_2020_2021 - (SELECT AVG(avg_net_added_2020_2021) FROM sa2_with_net_added)) /
    (SELECT STDDEV(avg_net_added_2020_2021) FROM sa2_with_net_added) AS z_score
FROM
    sa2_with_net_added);
"""))

Creating Z-value for population change and store it in a table called <u><strong> population_change_zscore </strong></u>.

In [ ]:
query(conn, "SELECT * FROM population_change_zscore;")

We also noticed an interesting trend between `mean income` and `population change` of various locations. We will discuss this further in <strong><u>section 6</u></strong> and in the  <strong><u>report</u></strong>.

## Step 4: Refined Bustling Metric

We have formed 2 new tables above, storing a z-values for `Opal Data` and `Population Change Data`. we can now use these additional parameters to calculate a more accurate bustling metric.

We store the data in a table called <u><strong> sigmoid_values_refined </strong></u>.

In [ ]:
conn.execute(text( """
DROP TABLE IF EXISTS sigmoid_values_refined;
CREATE TABLE sigmoid_values_refined
AS
(SELECT
    sa2_name21,
    1.0 / (1.0 + EXP(- (b.z_score + s.z_score + p.z_score + c.z_score + t.z_score + pc.z_score))) AS sigmoid_value
FROM
    businesses_zscore b
JOIN
    stops_zscore s USING(sa2_name21)
JOIN
    stops_zscore p USING(sa2_name21)
JOIN
    catchments_zscore c USING(sa2_name21)
JOIN
    tap_summary_per_sa2 t USING (sa2_name21)
JOIN
    population_change_zscore pc USING (sa2_name21));
"""))

We have a table which displays the `Refined Bustling Metric for all SA2 regions`.

In [ ]:
query(conn, """
SELECT sa2_name21 as Region, sigmoid_value AS bustling_metric
FROM sigmoid_values_refined
WHERE sigmoid_value IS NOT NULL
""")

## Step 5: Correlation Analysis

In this step, we will find a correlation between the `median income` and `bustling scores` for each region in our SA2 table.

We will use the data we imported into the `income` table in `1.7`. We will use `Pearson's correlation coefficient` to find the correlation.

In [ ]:
query(conn, """
SELECT
    (
        (SUM(s.sigmoid_value * CAST(i.median_income AS BIGINT)) - COUNT(*) * AVG(s.sigmoid_value) * AVG(CAST(i.median_income AS BIGINT))) /
        (SQRT((SUM(s.sigmoid_value * s.sigmoid_value) - COUNT(*) * AVG(s.sigmoid_value) * AVG(s.sigmoid_value)) *
              (SUM(CAST(i.median_income AS BIGINT) * CAST(i.median_income AS BIGINT)) - COUNT(*) * AVG(CAST(i.median_income AS BIGINT)) * AVG(CAST(i.median_income AS BIGINT)))))
    ) AS correlation_coefficient
FROM
    income_greater_sydney i
JOIN
    sigmoid_values s ON i.sa2_name = s.sa2_name21
""")

Here we can see a correlation of `-0.227188` which indicates a slight negative linear relationships `median income` between and `bustling scores`.

## Step 6: Graphs and Visualization

### 6.1 Interactive Map

This map shows the various regions of <strong>Greater Sydney</strong> and how bustling they are with `red` being *not bustling* and `green` being *bustling*

In [ ]:
conn.execute(text('''DROP TABLE IF EXISTS new_joined_table;
CREATE TABLE new_joined_table AS
SELECT
    svr.*,
    SA2_geom.geom
FROM
    sigmoid_values_refined svr
JOIN
    SA2_geom ON svr.sa2_name21 = SA2_geom.sa2_name21;
'''))
query = "SELECT * FROM new_joined_table"
gdf = gpd.read_postgis(query, conn, geom_col='geom')


colormap = cm.LinearColormap(colors=['red', 'green'], vmin=0, vmax=1)

def score_to_color(score):
    """Convert a score to a color, handling None or invalid values."""
    try:
        if score is None or pd.isna(score):
            return '#808080'
        return colormap(score)
    except:
        return '#808080'

m = folium.Map(location=[-33.8688, 151.2093], zoom_start=12)

geojson = folium.GeoJson(
    gdf,
    style_function=lambda feature: {
        'fillColor': score_to_color(feature['properties'].get('sigmoid_value')),
        'color': 'transparent',
        'weight': 0,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['sa2_name21', 'sigmoid_value'],
        aliases=['Region: ', 'Bustling Score: '],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=250,
    )
).add_to(m)
m

### 6.2 Graphs

In [ ]:
q = """
SELECT
    svr.sa2_name21 as name,
    svr.z_score as z_score,
    income.mean_income
FROM
    population_change_zscore svr
JOIN
    income ON svr.sa2_name21 = income.sa2_name;
"""

data = pd.read_sql(q, conn)


This graph shows the trend between `mean income` and `population change`. More details in <strong><u>report</u></strong>.

In [ ]:
data['mean_income'] = pd.to_numeric(data['mean_income'], errors='coerce')

data = data.dropna(subset=['mean_income'])

data_sorted = data.sort_values(by='mean_income')

fig = px.scatter(data_sorted, x='mean_income', y='z_score',
                 hover_data=['name'],
                 title='Scatter Plot of Z-Score vs Mean Income',
                 labels={'mean_income': 'Mean Income', 'z_score': 'Z-Score'})

fig.update_layout(xaxis=dict(range=[0, data_sorted['mean_income'].max()]))
fig.show()